In [7]:
%%sh

# Name of algo -> ECR
algorithm_name=sm-pretrained-pycaret

cd container

#make serve executable
chmod +x model/serve

account=$(aws sts get-caller-identity --query Account --output text)

# Region, defaults to us-west-2
region=$(aws configure get region)
region=${region:-us-east-1}

fullname="${account}.dkr.ecr.${region}.amazonaws.com/${algorithm_name}:latest"

# If the repository doesn't exist in ECR, create it.
aws ecr describe-repositories --repository-names "${algorithm_name}" > /dev/null 2>&1

if [ $? -ne 0 ]
then
    aws ecr create-repository --repository-name "${algorithm_name}" > /dev/null
fi

# Get the login command from ECR and execute it directly
aws ecr get-login-password --region ${region}|docker login --username AWS --password-stdin ${fullname}

# Build the docker image locally with the image name and then push it to ECR
# with the full name.

docker build  -t ${algorithm_name} .
docker tag ${algorithm_name} ${fullname}

docker push ${fullname}

Login Succeeded

Step 1/10 : FROM python:3.8
 ---> 274827c0462d
Step 2/10 : RUN apt-get -y update && apt-get install -y --no-install-recommends          wget          python3          nginx          ca-certificates     && rm -rf /var/lib/apt/lists/*
 ---> Using cache
 ---> c5c18630991a
Step 3/10 : RUN wget https://bootstrap.pypa.io/get-pip.py && python3 get-pip.py &&     pip install flask gevent gunicorn &&         rm -rf /root/.cache
 ---> Using cache
 ---> a9948b3789ab
Step 4/10 : RUN pip install pycaret
 ---> Using cache
 ---> 1ed6a3b8e77d
Step 5/10 : RUN pip install pandas
 ---> Using cache
 ---> 9b748b3d72e1
Step 6/10 : ENV PYTHONUNBUFFERED=TRUE
 ---> Using cache
 ---> 0d9ee0689bd1
Step 7/10 : ENV PYTHONDONTWRITEBYTECODE=TRUE
 ---> Using cache
 ---> b18a6cda2b18
Step 8/10 : ENV PATH="/opt/program:${PATH}"
 ---> Using cache
 ---> 9d76b87618aa
Step 9/10 : COPY model /opt/program
 ---> Using cache
 ---> af31aa48d668
Step 10/10 : WORKDIR /opt/program
 ---> Using cache
 ---> 118d7e8cc7

WARNING! Your password will be stored unencrypted in /home/ec2-user/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store



In [17]:
import boto3
from sagemaker import get_execution_role

sm_client = boto3.client(service_name='sagemaker')
runtime_sm_client = boto3.client(service_name='sagemaker-runtime')

account_id = boto3.client('sts').get_caller_identity()['Account']
region = boto3.Session().region_name


role = get_execution_role()

In [18]:
from time import gmtime, strftime

model_name = 'pycaretModel-' + strftime("%Y-%m-%d-%H-%M-%S", gmtime())
container = '{}.dkr.ecr.{}.amazonaws.com/sm-pretrained-pycaret:latest'.format(account_id, region)
instance_type = 'm4.xlarge'

print('Model name: ' + model_name)
print('Container image: ' + container)

container = {
    'Image': container
}

create_model_response = sm_client.create_model(
    ModelName = model_name,
    ExecutionRoleArn = role,
    Containers = [container])

print("Model Arn: " + create_model_response['ModelArn'])

Model name: pycaretModel-2022-07-14-14-59-55
Container image: 348305232947.dkr.ecr.us-east-1.amazonaws.com/sm-pretrained-pycaret:latest
Model Arn: arn:aws:sagemaker:us-east-1:348305232947:model/pycaretmodel-2022-07-14-14-59-55


In [19]:
instance_type = 'ml.m4.xlarge'
endpoint_config_name = 'pycaret-config' + strftime("%Y-%m-%d-%H-%M-%S", gmtime())
print('Endpoint config name: ' + endpoint_config_name)

create_endpoint_config_response = sm_client.create_endpoint_config(
    EndpointConfigName = endpoint_config_name,
    ProductionVariants=[{
        'InstanceType': instance_type,
        'InitialInstanceCount': 1,
        'InitialVariantWeight': 1,
        'ModelName': model_name,
        'VariantName': 'AllTraffic'}])

print("Endpoint config Arn: " + create_endpoint_config_response['EndpointConfigArn'])

Endpoint config name: pycaret-config2022-07-14-14-59-58
Endpoint config Arn: arn:aws:sagemaker:us-east-1:348305232947:endpoint-config/pycaret-config2022-07-14-14-59-58


In [20]:
%%time

import time

endpoint_name = 'pycaret-endpoint' + strftime("%Y-%m-%d-%H-%M-%S", gmtime())
print('Endpoint name: ' + endpoint_name)

create_endpoint_response = sm_client.create_endpoint(
    EndpointName=endpoint_name,
    EndpointConfigName=endpoint_config_name)
print('Endpoint Arn: ' + create_endpoint_response['EndpointArn'])

resp = sm_client.describe_endpoint(EndpointName=endpoint_name)
status = resp['EndpointStatus']
print("Endpoint Status: " + status)

print('Waiting for {} endpoint to be in service...'.format(endpoint_name))
waiter = sm_client.get_waiter('endpoint_in_service')
waiter.wait(EndpointName=endpoint_name)

Endpoint name: pycaret-endpoint2022-07-14-15-00-01
Endpoint Arn: arn:aws:sagemaker:us-east-1:348305232947:endpoint/pycaret-endpoint2022-07-14-15-00-01
Endpoint Status: Creating
Waiting for pycaret-endpoint2022-07-14-15-00-01 endpoint to be in service...
CPU times: user 131 ms, sys: 3.23 ms, total: 134 ms
Wall time: 5min 1s


In [21]:
import json
content_type = "application/json"
request_body = {
	"user":{
		"user_segment": 0
	},
	"transaction":{
		"id": 133,
		"producto": "CB",
		"movilizado": 34990.0
	}
}
#Serialize data for endpoint
data = json.loads(json.dumps(request_body))
payload = json.dumps(data)

#Endpoint invocation
response = runtime_sm_client.invoke_endpoint(
    EndpointName=endpoint_name,
    ContentType=content_type,
    Body=payload)

#Parse results
result = json.loads(response['Body'].read().decode())
result

{'0': {'id': 133,
  'producto': 'CB',
  'movilizado': 34990.0,
  'Label': 1,
  'Score': 0.6524}}